In [1]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Draw a Shapes

Attempt at programmatically drawing shapes.

All units in mm. ```1``` = ```1 mm```.

In [2]:
s = GCode.Shapes.Square()


In [3]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [4]:
cnc.status


'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [16]:
cnc.home()
cnc.status


'<Idle|MPos:-259.000,-174.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [17]:
o0 = 10


def origin_calc(lines):
    if len(lines) == 0:
        o = o0
    else:
        o = np.sum(list(map(lambda s: s.len_side, lines))) + o0
    return np.array([o, o])


In [18]:
sides = [0.1, 0.25, 0.5, 0.75, 1, 2.5, 5, 10, 20, 25.4, 25.4 / 4]
np.sum(sides) + o0


81.85

In [19]:
lines = list()
for side in sides:
    _ = GCode.Shapes.Square(len_side=side, origin=origin_calc(lines))
    lines.append(_)


In [20]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


[Square<O=[10 10], L=0.1>,
 Square<O=[10.1 10.1], L=0.25>,
 Square<O=[10.35 10.35], L=0.5>,
 Square<O=[10.85 10.85], L=0.75>,
 Square<O=[11.6 11.6], L=1>,
 Square<O=[12.6 12.6], L=2.5>,
 Square<O=[15.1 15.1], L=5>,
 Square<O=[20.1 20.1], L=10>,
 Square<O=[30.1 30.1], L=20>,
 Square<O=[50.1 50.1], L=25.4>,
 Square<O=[75.5 75.5], L=6.35>]

In [21]:
prog = GCode.Program(lines=lines)

prog.generate_gcode()


In [27]:
cnc.run(prog)


90.1433253288269